<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       Resume Classification Model Training using Vantage In-Db Functions
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>
<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>Introduction</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>This notebook is a part of the Resume Analyzer application.
In this notebook we will use data from a collection of Resume Examples taken from livecareer.com for categorizing a given resume into any of the labels defined in the dataset. Resumes are differentiated into their respective labels with filename as the id defined in the csv. Data was obtained by scrapping individual resume examples from www.livecareer.com website.</p>
<br>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>The Data used for model training consists of:</b>
<li style = 'font-size:14px;font-family:Arial;color:#00233C'><code>ID:</code> Unique identifier and file name for the respective pdf. </li>
    <li style = 'font-size:14px;font-family:Arial;color:#00233C'><code>Resume_str:</code> Contains the resume text only in string format.</li>
<li style = 'font-size:14px;font-family:Arial;color:#00233C'><code>Category:</code> Category of the job the resume was used to apply.</li>
</p>

<p></p>    
<br>  
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Present categories are: HR, Designer, Information-Technology, Teacher, Advocate, Business-Development, Healthcare, Fitness, Agriculture, BPO, Sales, Consultant, Digital-Media, Automobile, Chef, Finance, Apparel, Engineering, Accountant, Construction, Public-Relations, Banking, Arts, Aviation   

</p>



<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>1. Connect to Vantage</b>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Here, we import the required libraries, set environment variables and environment paths (if required).</p>

In [ ]:
import os
import warnings
warnings.filterwarnings('ignore')


from teradataml import *

# Modify the following to match the specific client environment settings
display.max_rows = 5


<p style = 'font-size:16px;font-family:Arial;color:#00233C'>You will be prompted to provide the password. Enter your password, press the Enter key, and then use the down arrow to go to the next cell.</p>

In [ ]:
%run -i ../UseCases/startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Setup for execution of notebook. Begin running steps with Shift + Enter keys.</p>

In [ ]:
%%capture
execute_sql('''SET query_band='DEMO=PP_Resume_Classification_model_training.ipynb;' UPDATE FOR SESSION; ''')

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Getting Data for This Demo</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We have provided data for this demo on cloud storage. You can either run the demo using foreign tables to access the data without any storage on your environment or download the data to local storage, which may yield faster execution. Still, there could be considerations of available storage. Two statements are in the following cell, and one of them is commented out. You may switch between the modes by changing the comment string.</p>

In [ ]:
%run -i ../UseCases/run_procedure.py "call get_data('DEMO_ResumeClassification_local');"           # Takes 1 minute

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Optional step – if you want to see status of databases/tables created and space used.</p>


In [ ]:
%run -i ../UseCases/run_procedure.py "call space_report();"        # Takes 10 seconds

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>2. Explore the dataset</b>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The dataset consists of text from from various resumes with the id and category of the resumes.</p>

In [ ]:
tdf = DataFrame(in_schema("DEMO_ResumeClassification", "Resume_data"))
print("Shape of the data: ", tdf.shape)
tdf

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>3. Check model table</b>
<br>
</p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Before starting the process we will check if the model table already exists
<br>
</p>

In [ ]:
from IPython.display import display, Markdown
def display_msg(msg):
    return display(Markdown(
        f"""<div class="alert alert-block alert-info">
        <p style='font-size:20px;font-family:Arial;color:#00233C'><b>Note: </b><i>{msg}</i></p>"""))

In [ ]:
model_df=DataFrame.from_query('''select tablename from dbc.tablesv where databasename = 'demo_user' and tablename = 'resume_category_model_tb';''')
if model_df.shape[0]==1:
    display_msg('Model table exists. No need to execute the below steps')
else:
    display_msg('Please continue with the below steps and create the model table')

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>4. Model Creation</b>
<br>
</p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Model table should exist in order to do Resume classification, We can either create the model table from raw data, or load the model table from the cloud
<br>
</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Note: If we want to load the model table from cloud please enter "no". If we want to create the model table please enter "yes" and follow the steps below"
</b>
</p>

In [ ]:
# import time
# Request user's input
generate = input("Do you create a new model? ('yes'/'no'): ")

# Check the user's input
if generate.lower() == 'no':
    print("\nGreat! We'll load the model table from cloud.")

    print("\nLoading model table from cloud, please wait...")
    # start = time.time()
    qry="""CREATE MULTISET TABLE DEMO_USER.resume_category_model_tb 
     (
      token VARCHAR(37) CHARACTER SET UNICODE NOT CASESPECIFIC,
      category VARCHAR(100) CHARACTER SET UNICODE NOT CASESPECIFIC,
      prob FLOAT)
    NO PRIMARY INDEX;"""
    
    qry1="""Insert into resume_category_model_tb 
        Select "token", "category", "prob" from DEMO_ResumeClassification_db.Resume_Model_Data;"""

    try:
        execute_sql(qry)
        execute_sql(qry1)
        # end = time.time()
        print('Table Loaded')
        # print(end-start)
        
    except:
        db_drop_table('resume_category_model_tb')
        execute_sql(qry)
        execute_sql(qry1)
        # end = time.time()
        print('Table Loaded')
        # print(end-start)


    print("\nModel table loaded successfully!")

elif generate.lower() == 'yes':
    print("\nTo create the model again you will have to execute all steps below")
    
else:
    print("\nInvalid input. Please enter 'yes' or 'no' to proceed.")

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>5. Text Parsing</b>
<br>
</p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>A text parser, also known as a text tokenizer, is a software component that breaks a text into its constituent parts, such as words, phrases, sentences, or other meaningful units. Text parsing is an important technique in natural language processing (NLP) and is used in a wide range of applications, from search engines and chatbots to email filters and data analysis tools.
<br>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The TD_TextParser performs the following operations:
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>Tokenizes the text in the specified column</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>Removes the punctuations from the text and converts the text to lowercase</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>Removes stop words from the text and converts the text to their root forms</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>Creates a row for each word in the output table</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>Performs stemming; that is, the function identifies the common root form of a word by removing or replacing word suffixes</li></p>
<br>
</p>

In [ ]:
qry = '''CREATE MULTISET TABLE tfidf_input_tokenized AS (
    SELECT ResID,
        CAST(token AS VARCHAR(15)) AS token,
        category
        FROM TD_TextParser (
        ON DEMO_ResumeClassification.Resume_data AS InputTable
        USING
        TextColumn ('Resume_str')
        ConvertToLowerCase ('true')
        OutputByWord ('true')
        Punctuation ('\[.,?\!\]')
        RemoveStopWords ('true')
        StemTokens ('true')
        Accumulate('ResID', 'category') 
        ) AS dt ) WITH DATA Primary Index(ResID);'''

try:
    execute_sql(qry)
except:
    db_drop_table("tfidf_input_tokenized")
    execute_sql(qry)

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>6. TF-IDF</b>
<br>
</p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Term Frequency-Inverse Document Frequency (TF-IDF) is a technique for evaluating the importance of a specific term in a specific document in a document set. Term frequency (tf) is the number of times that the term appears in the document and inverse document frequency (idf) is the number of times that the term appears in the document set. The TF-IDF score for a term is tf * idf. A term with a high TF-IDF score is relevant to the specific document.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We can use the TF-IDF scores as input for documents clustering and classification algorithms, including:
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>Cosine-similarity</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>Latent Dirichlet allocation</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>K-means clustering</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>K-nearest neighbors</li></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>TD_TFIDF function represents each document as an N-dimensional vector, where N is the number of terms in the document set (therefore, the document vector is sparse). Each entry in the document vector is the TF-IDF score of a term.</p>
<br>
</p>

In [ ]:
qry = '''CREATE MULTISET TABLE resume_tf_idf_train AS ( SELECT *
            FROM TD_TFIDF(
            ON tfidf_input_tokenized AS InputTable
            USING
            DocIdColumn ('ResID')
            TokenColumn ('token')
            TFNormalization ('LOG')
            IDFNormalization ('SMOOTH')
            Regularization ('L2')
            Accumulate('category')
            ) AS dt ) WITH DATA Primary Index(ResID);'''
try:
    execute_sql(qry)
except:
    db_drop_table("resume_tf_idf_train")
    execute_sql(qry)

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>7. Naive Bayes Text Classifier</b>
<br>
</p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The TD_NaiveBayesTextClassifierTrainer function calculates the conditional probabilities for token-category pairs, the prior probabilities, and the missing token probabilities for all categories. The trainer function trains the model with the probability values, and the predict function uses the values to classify documents into categories.</p>

<br>
</p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Note: The below query build model table which will take approximately 30 minutes</b></p>

In [ ]:
qry = '''SELECT TOP 1 'Model table created' as output
FROM TD_NaiveBayesTextClassifierTrainer(	
ON resume_tf_idf_train AS InputTable	
OUT PERMANENT  TABLE ModelTable (resume_category_model_tb)	
USING	
TokenColumn ('token')	
ModelType ('Multinomial')	
DocCategoryColumn ('category')	
) AS dt;'''

try:
    execute_sql(qry)
except:
    db_drop_table("resume_category_model_tb")
    execute_sql(qry)
    
print("Model table Created")    

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>8. Cleanup</b>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Work Tables</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Cleanup work tables to prevent errors next time. This section drops all the tables created during the demonstration.</p>

In [ ]:
try:
    db_drop_table("resume_tf_idf_train")
    db_drop_table("tfidf_input_tokenized")
except:
    pass
    

<div class="alert alert-block alert-danger">
<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Note: <i>If you do not want the model table anymore and do not want to do Resume Classification from the application than go ahead and drop the table using the cell below, else do not drop the model table.</i></b></p>
</div>

In [ ]:
try:
    db_drop_table("resume_category_model_tb")
    
except:
    pass

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'> <b>Databases and Tables </b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The following code will clean up tables and databases created above.</p>

In [ ]:
%run -i ../UseCases/run_procedure.py "call remove_data('DEMO_ResumeClassification');"         -- Takes 5 seconds

<footer style="padding-bottom:35px; background:#f9f9f9; border-bottom:3px solid #00233C">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2024. All Rights Reserved
        </div>
    </div>
</footer>